In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(legacy='1.25')

# Comparing Movies in different streaming services
* Find out which genre of movies each country creates the most

# Cleaning Data

In [69]:
netflix = pd.read_csv("netflix_titles.csv", index_col="show_id")
amazon = pd.read_csv("amazon_prime_titles.csv", index_col="show_id")
disney = pd.read_csv("disney_plus_titles.csv", index_col="show_id")

In [61]:
# Drop rows with missing values in country and listed_in
def drop_values(platform):
    platform.dropna(subset=["country", "listed_in"], inplace=True)
drop_value(netflix)
drop_value(amazon)
drop_value(disney)

In [62]:
# Removing any TV show type
netflix = netflix[netflix["type"] == "Movie"]
amazon = amazon[amazon["type"] == "Movie"]
disney = disney[disney["type"] == "Movie"]

In [66]:
# Focusing on Necessary Columns
netflix = netflix[["title", "country", "listed_in"]]
amazon = amazon[["title", "country", "listed_in"]]
disney = disney[["title", "country", "listed_in"]]

In [68]:
disney

,title,country,listed_in
show_id,,,
s3,Ice Age: A Mammoth Christmas,United States,"Animation, Comedy, Family"
s6,Becoming Cousteau,United States,"Biographical, Documentary"
s10,A Muppets Christmas: Letters To Santa,United States,"Comedy, Family, Musical"
s13,The Pixar Story,United States,"Documentary, Family"
s20,Enchanted,United States,"Comedy, Family, Fantasy"
...,...,...,...
s1446,X-Men Origins: Wolverine,"United States, United Kingdom","Action-Adventure, Family, Science Fiction"
s1447,Night at the Museum: Battle of the Smithsonian,"United States, Canada","Action-Adventure, Comedy, Family"
s1448,Eddie the Eagle,"United Kingdom, Germany, United States","Biographical, Comedy, Drama"
